In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
# STEP 1
# IMPORT SIMON P&L2022

import pandas as pd
import numpy as np

#import the file with raw data
simonProfitAndLoss_df = pd.read_csv('Data/Simon2022_ProfitAndLoss.csv', sep=";", header = None, skiprows=[0],names=["Date", "PaL", "PaL Sum"])

simonProfitAndLoss_df['Date'] = pd.to_datetime(simonProfitAndLoss_df['Date'], infer_datetime_format=True)


#add id to each row starting from 0
#simonProfitAndLoss_df['id'] = pd.RangeIndex(0, len(simonProfitAndLoss_df)) + 1

#replace "," for "." for conversion
simonProfitAndLoss_df['PaL'] = simonProfitAndLoss_df['PaL'].apply(lambda x: float(x.split()[0].replace(',', '.')))
#convert to integer
simonProfitAndLoss_df['PaL'] = simonProfitAndLoss_df['PaL'].astype(int)

#replace "," for "." for conversion
simonProfitAndLoss_df['PaL Sum'] = simonProfitAndLoss_df['PaL Sum'].apply(lambda x: float(x.split()[0].replace(',', '.')))
#convert to integer
simonProfitAndLoss_df['PaL Sum'] = simonProfitAndLoss_df['PaL Sum'].astype(int)
simonProfitAndLoss_df = simonProfitAndLoss_df.drop('PaL Sum', axis=1)


#print dataframe
#with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):print(simonProfitAndLoss_df)



In [3]:
# STEP 2
# CREATE WITHDRAW BALANCES AND MERGE THEM WITH MAIN DATAFRAME



import pandas as pd
import numpy as np

withdraw_df = pd.DataFrame(np.array([['2022-04-05', -150000], 
                                     ['2022-05-06', -200000],
                                     ['2022-08-01',-100000],
                                     ['2022-09-08',-100000],
                                     ['2022-09-23',-100000],
                                     ['2022-09-29',-250000],
                                     ['2022-10-17',-250000],
                                     ['2022-11-08',-250000]]),columns=['Date', 'Withdraw'])


withdraw_df['Date'] = pd.to_datetime(withdraw_df.Date)

withdraw_df['Withdraw'] = withdraw_df['Withdraw'].astype(int)

newdate1 = simonProfitAndLoss_df['Date'][0]

newdate = withdraw_df['Withdraw'][0]

df = pd.merge(simonProfitAndLoss_df, withdraw_df, how='outer', on='Date')

df['Withdraw'] = df['Withdraw'].fillna(0)

df ['PaLAndWithdraw'] = df ['PaL'] + df ['Withdraw']


#starting balance at the beginning of 2022
startingBalance2022 = 499418
df['PaL Sum'] = df['PaLAndWithdraw'].cumsum() + startingBalance2022

#PaL sum without withdraw
df['PaL SumWW'] = df['PaL'].cumsum() + startingBalance2022


simonProfitAndLoss_df = df

#print dataframe
#with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):print(simonProfitAndLoss_df)



In [4]:
#STEP 3
#CALCULATE ADDITIONAL VARIABLES: PctDif / PaL Abv and Blw Count / CumMaxCount

#%difference
simonProfitAndLoss_df['PctDif'] = (simonProfitAndLoss_df['PaL Sum'] / simonProfitAndLoss_df['PaL Sum'].shift().astype(float)-1)*100
pd.options.display.float_format = '{:.2f}'.format

simonProfitAndLoss_df['PctDifCumSum'] = simonProfitAndLoss_df['PctDif'].cumsum()

simonProfitAndLoss_df['PctDifWW'] = (simonProfitAndLoss_df['PaL SumWW'] / simonProfitAndLoss_df['PaL SumWW'].shift().astype(float)-1)*100
pd.options.display.float_format = '{:.2f}'.format

simonProfitAndLoss_df['PctDifCumSumWW'] = simonProfitAndLoss_df['PctDifWW'].cumsum()

#define streak, streak is defined as consecutive 
#define count above and below
simonProfitAndLoss_df['PaLAbv'] = np.where(simonProfitAndLoss_df['PaL'] > 0, 1, 0)
simonProfitAndLoss_df['PaLBlw'] = np.where(simonProfitAndLoss_df['PaL'] < 0, 1, 0)

#cumsum cumcount above and below
simonProfitAndLoss_df['PaLAbvCount'] = simonProfitAndLoss_df.groupby(simonProfitAndLoss_df['PaLAbv'].eq(0).cumsum()).cumcount()
simonProfitAndLoss_df['PaLBlwCount'] = simonProfitAndLoss_df.groupby(simonProfitAndLoss_df['PaLBlw'].eq(0).cumsum()).cumcount()


#identify cumulative interim maximum to identify peaks
simonProfitAndLoss_df ['CumMax'] = simonProfitAndLoss_df['PaL Sum'].cummax()
                 
#now find blocks that are higher then previous, mark them as 1 and then count the day difference. If 1, calc the date difference
simonProfitAndLoss_df['CumMaxTrue'] = np.where(simonProfitAndLoss_df['CumMax'] > simonProfitAndLoss_df['CumMax'].shift(), 0, 1 )

#now just count how many sessions it took to reach to new highs
simonProfitAndLoss_df['CumMaxTrueCount'] = simonProfitAndLoss_df.groupby(simonProfitAndLoss_df['CumMaxTrue'].eq(0).cumsum()).cumcount()

    
    
#experimental block with useful variables
idvar = 10

idvar_pal = simonProfitAndLoss_df.loc[idvar]['PaL']
idvar_PaLAbvCount = simonProfitAndLoss_df.loc[idvar]['PaLAbvCount'] 
idvar_PaLBlwCount = simonProfitAndLoss_df.loc[idvar]['PaLBlwCount'] 

if (idvar_PaLAbvCount > idvar_PaLBlwCount):
    streak = "Up"
else: 
    streak = "Down"
        
#median = mad(simonProfitAndLoss_df['PaL'], c=1)
averagewin = int (simonProfitAndLoss_df.loc[simonProfitAndLoss_df['PaL'] > 0, 'PaL'].mean())
averageloss = int(simonProfitAndLoss_df.loc[simonProfitAndLoss_df['PaL'] < 0, 'PaL'].mean())
maxloss = int (simonProfitAndLoss_df['PaL'].min())
maxwin = int (simonProfitAndLoss_df['PaL'].max())


#print experimental values
#print (idvar_pal, idvar_PaLAbvCount, idvar_PaLBlwCount, streak, median, averagewin, averageloss, maxloss, maxwin)


#print dataframe
#with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):print(simonProfitAndLoss_df)



In [5]:
#STEP 4
#CREATE CPI AND NFP DATAFRAME

import pandas as pd
import numpy as np

economicData_df = pd.DataFrame(np.array([
                                     ['2022-01-12', "CPI","Economic data"], 
                                     ['2022-02-10', "CPI","Economic data"],
                                     ['2022-03-10', "CPI","Economic data"],
                                     ['2022-04-12', "CPI","Economic data"],
                                     ['2022-05-11', "CPI","Economic data"],
                                     ['2022-06-10', "CPI","Economic data"],
                                     ['2022-07-13', "CPI","Economic data"],
                                     ['2022-08-10', "CPI","Economic data"],
                                     ['2022-09-13', "CPI","Economic data"],
                                     ['2022-10-13', "CPI","Economic data"],
                                     ['2022-11-10', "CPI","Economic data"],
                                     ['2022-01-07', "NFP","Economic data"], 
                                     ['2022-02-04', "NFP","Economic data"],
                                     ['2022-03-04', "NFP","Economic data"],
                                     ['2022-04-01', "NFP","Economic data"],
                                     ['2022-05-06', "NFP","Economic data"],
                                     ['2022-06-03', "NFP","Economic data"],
                                     ['2022-07-08', "NFP","Economic data"],
                                     ['2022-08-05', "NFP","Economic data"],
                                     ['2022-09-02', "NFP","Economic data"],
                                     ['2022-10-07', "NFP","Economic data"],
                                     ['2022-11-04', "NFP","Economic data"]]),columns=['Date', 'News', 'Event'])


economicData_df['Date'] = pd.to_datetime(economicData_df.Date)

#print (economicData_df)

In [6]:
#STEP 5
#IMPORT NEWS CSV WITH CENTRAL BANK EVENTS


import pandas as pd
import numpy as np

#import the file with raw data
#EcoData
#CentralBanks2022
news_df = pd.read_csv('Data/CentralBanks2022.csv', sep=";", header = None, skiprows=[0],names=["Date", "News"])


#convert string time to datetime
#introduce infer_datetime_format variable in pd.to_datetime
news_df['Date'] = pd.to_datetime(news_df['Date'], infer_datetime_format=True)
news_df['Event'] = "Central Bank"


newDf = pd.concat([news_df, economicData_df])


news_df = newDf.reset_index(drop=True)


#print (news_df)

In [7]:
#STEP 6
#Merge SimonDF and NewsDf

merged_df = pd.merge (simonProfitAndLoss_df,news_df, how='outer', on='Date')

#print dataframe
#with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):print(simonProfitAndLoss_df)



In [8]:
#STEP 7
#Build visual representation of all the data

import pandas as pd
from math import pi
from bokeh.layouts import column
from datetime import datetime as dt
from bokeh.io import output_notebook
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import ColumnDataSource
import numpy as np
from bokeh.models import HoverTool


from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap

# prepare some data for location of the circle
y3 = [-1, -1, -1]

def showDataframe (dataframe):

    
    #defining input dataframe as a source so that hoover tooltip works
    source = ColumnDataSource(dataframe)
    
    #creating new column color based on which the color is then selected
    dataframe['color'] = np.where(dataframe['PaL'] > 0, 'green', 'red')
    
    #creating better ratio between prices and the P&L curve
    dataframe['PaLSumDivided'] = dataframe['PaL Sum'] / 10
    
    #hacking the dataframe by changing the integer to string :)
    dataframe['PaLSumString'] = dataframe['PaL Sum'].astype(str)
    
    dataframe['DateToString'] = dataframe['Date'].astype(str)
    
    #create index for future search
    dataframe['Index'] = dataframe.index
    
    #print dataframe
    #print (dataframe)

    
    #adding hover capability    
    hover = HoverTool(
    tooltips = [
        ("Index", "@Index"),
        ("Date", "@DateToString"),
        ("P&L", "@PaL"),
        ("Sum", "@PaLSumString"),
        ("News", "@News"),
        
    ]
    )
    
    
    hoverPct = HoverTool(
    tooltips = [
        ("Index", "@Index"),
        ("Date", "@DateToString"),
        ("P&L", "@PaL"),
        ("%DifW", "@PctDifCumSum"),
        ("%DifWW", "@PctDifCumSumWW"),
        
    ]
    )
    
    

    #create new dataframe that does not containt NaN rows
    dfWithoutNews = dataframe[dataframe['News'].notna()]
    dfWithoutNews['DateToString'] = dfWithoutNews['Date'].astype(str)
    
    #create dataframe that consists only certain rows, so that I can plot it in bokeh, need x = date, and y = PaL
    dfNewsEconomicEvents = dfWithoutNews[['Date','DateToString','PaLSumString','Index', 'PaL', 'News','Event']].copy()
    
    dfNewsCentralBank = dfWithoutNews[['Date','DateToString','PaLSumString','Index', 'PaL', 'News','Event']].copy()
    
    #move PaL higher so that it does not colide in location with previous news
    dfNewsEconomicEvents ['PaL'] = dfNewsEconomicEvents ['PaL']  + 10000
    
    #filter only data that have Economic character like CPI
    dfNewsEconomicEvents1 = dfNewsEconomicEvents[dfNewsEconomicEvents['Event'].str.startswith('Economic data', na=False)]
    dfNewsCentralBank1 = dfNewsCentralBank[dfNewsCentralBank['Event'].str.startswith('Central Bank', na=False)]
    

    #with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 3,):print(dataframe)


    
    #introduce fiture where tooltips define tooltip
    c1 = figure(title="P&L Plot", x_axis_label="Date", y_axis_label="P&L",width=1000, tools=[hover])
               #tooltips=TOOLTIPS)
   
    #disable scientific fucking of my y scale
    c1.left[0].formatter.use_scientific = False

    
    #width 24*60*60*1000

    c1.vbar(x = dataframe.Date,top = dataframe['PaL'], width=1, bottom=0, color=dataframe['color'])

    
    #print news on c1 canvas
    c1.circle(source = dfNewsCentralBank1, x='Date', y='PaL', color="blue", size=5)
    c1.square(source = dfNewsEconomicEvents1, x='Date', y=0, color="red", size=5)
    
    
    
    c1.line(x='Date', y='PaLSumDivided', line_color='blue', line_width=2, source=source)
    
    #p.triangle(source = dfWithVIX, x='Date', y='PL', color="red", size=15)

    

    c1.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
  

    
    c1.ray(x=dataframe.Date, y=[150000], length=0, angle=0, line_width=1, color='blue')
    c1.ray(x=dataframe.Date, y=[-150000], length=0, angle=0, line_width=1, color='red')
    
    
    c2 = figure(title="Positive Streak", y_axis_label="PositiveCount",width=1000,height=200, tools=[hover])
    c2.vbar(x = dataframe.Date,top = dataframe['PaLAbvCount'], width=1, bottom=0, color='green')
    c2.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
    
    
    c3 = figure(title="Negative Streak", y_axis_label="NegativeCount",width=1000,height=200, tools=[hover])
    c3.vbar(x = dataframe.Date,top = dataframe['PaLBlwCount'], width=1, bottom=0, color='red')
    c3.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
    
    
    c4 = figure(title="% Difference", y_axis_label="%",width=1000,height=200, tools=[hover])
    c4.vbar(x = dataframe.Date,top = dataframe['PctDif'], width=1, bottom=0, color=dataframe['color'])
    c4.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
    
    c6 = figure(title="% Return", x_axis_label="Date", y_axis_label="% Return",width=1000, tools=[hoverPct])
    c6.line(x='Date', y='PctDifCumSumWW', line_color='blue', line_width=2, source=source)
    c6.line(x='Date', y='PctDifCumSum', line_color='red', line_width=2, source=source)
    c6.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
    
    c5 = figure(title="Sessions To New Peak", y_axis_label="NumberOfSessions",width=1000,height=200, tools=[hover])
    c5.vbar(x = dataframe.Date,top = dataframe['CumMaxTrueCount'], width=1, bottom=0, color='purple')
    c5.xaxis.formatter=DatetimeTickFormatter(
            hours=["%b %Y"],
            days=["%b %Y"],
            months=["%b %Y"],
            years=["%b %Y"],
        )
    

    output_notebook() 

    show(column(c1, c2, c3, c4, c6, c5))

In [10]:
#run a code without warnings

import warnings
warnings.filterwarnings("ignore")

#showDataframe (merged_df)



import ipywidgets as widgets

get_data_button = widgets.Button(description='Compose Chart')

showDataframe (merged_df)

def getSpreadsheetApiData(b):
    showDataframe (merged_df)
    

# DISPLAY BUTTON
get_data_button.on_click(getSpreadsheetApiData)
display(get_data_button)

Loading BokehJS ...

Button(description='Compose Chart', style=ButtonStyle())